In [1]:
from ultralytics import YOLO
import json
import os
import shutil
import random
import cv2
import matplotlib.pyplot as plt

In [2]:
json_path = "annotations.json"
images_dir = "images"
labels_dir = "labels"
data_yaml = "data.yaml"
train_dir = "dataset/train"
val_dir = "dataset/val"
os.makedirs(train_dir + "/images", exist_ok=True)
os.makedirs(train_dir + "/labels", exist_ok=True)
os.makedirs(val_dir + "/images", exist_ok=True)
os.makedirs(val_dir + "/labels", exist_ok=True)

In [3]:
# Load JSON annotation file
with open(json_path, "r") as f:
    data = json.load(f)

file_names = data["train"]["file_names"]
rois_list = data["train"]["rois_list"]
occupancy_list = data["train"]["occupancy_list"]

# Split dataset (80% train, 20% val)
data_pairs = list(zip(file_names, rois_list, occupancy_list))
random.shuffle(data_pairs)
split_idx = int(0.8 * len(data_pairs))
train_data = data_pairs[:split_idx]
val_data = data_pairs[split_idx:]

In [4]:
# Function to process and save labels
def save_labels(file_name, rois, occupancy, split):
    label_path = f"dataset/{split}/labels/" + file_name.replace(".JPG", ".txt")
    img_path = os.path.join(images_dir, file_name)
    shutil.copy(img_path, f"dataset/{split}/images/")
    
    with open(label_path, "w") as lf:
        for obj, occupied in zip(rois, occupancy):
            x_values = [p[0] for p in obj]
            y_values = [p[1] for p in obj]
            x_center = sum(x_values) / len(x_values)
            y_center = sum(y_values) / len(y_values)
            width = max(x_values) - min(x_values)
            height = max(y_values) - min(y_values)
            class_id = 0 if occupied else 1
            lf.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Process all data
for file_name, rois, occupancy in train_data:
    save_labels(file_name, rois, occupancy, "train")
for file_name, rois, occupancy in val_data:
    save_labels(file_name, rois, occupancy, "val")

In [5]:
# Write data.yaml
with open(data_yaml, "w") as f:
    f.write("train: dataset/train\n")
    f.write("val: dataset/val\n")
    f.write("nc: 2\n")
    f.write("names: ['car', 'empty']\n")

In [6]:
# Load model
model = YOLO("yolo11n.pt")

# Train model
results = model.train(
    data=data_yaml, 
    epochs=5, 
    imgsz=640
)

# Change path in C:\Users\[user]\AppData\Roaming\Ultralytics\settings.json to where the "ML Model" folder is stored on your computer

Ultralytics 8.3.81  Python-3.13.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8700K 3.70GHz)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train24, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

train: Scanning C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\train\labels... 184 images, 0 backgrounds, 0 corrupt: 100%|██████████| 184/184 [00:00<00:00, 189.63it/s]


train: New cache created: C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\train\labels.cache


val: Scanning C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\val\labels... 47 images, 0 backgrounds, 0 corrupt: 100%|██████████| 47/47 [00:00<00:00, 226.38it/s]


val: New cache created: C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\val\labels.cache
Plotting labels to runs\detect\train24\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train24
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        5/5         0G      1.964      2.175      1.405        480        640: 100%|██████████| 12/12 [01:54<00:00,  9.56s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:12<00:00,  6.33s/it]

                   all         47       1472      0.944      0.151      0.341      0.171



5 epochs completed in 0.148 hours.
Optimizer stripped from runs\detect\train24\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train24\weights\best.pt, 5.5MB

Validating runs\detect\train24\weights\best.pt...
Ultralytics 8.3.81  Python-3.13.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8700K 3.70GHz)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:09<00:00,  4.55s/it]


                   all         47       1472      0.944      0.151      0.341      0.171
                   car         47        765      0.889      0.302      0.583      0.281
                 empty         44        707          1          0     0.0988     0.0608
Speed: 2.8ms preprocess, 93.7ms inference, 0.0ms loss, 23.8ms postprocess per image
Results saved to runs\detect\train24


In [7]:
# Visualize sample images
def visualize_samples(split, num=5):
    sample_images = random.sample(os.listdir(f"dataset/{split}/images"), num)
    for img_file in sample_images:
        img_path = f"dataset/{split}/images/{img_file}"
        label_path = f"dataset/{split}/labels/{img_file.replace('.JPG', '.txt')}"
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        with open(label_path, "r") as lf:
            labels = lf.readlines()
            for label in labels:
                class_id, x, y, w, h = map(float, label.split())
                h_img, w_img, _ = img.shape
                x1 = int((x - w/2) * w_img)
                y1 = int((y - h/2) * h_img)
                x2 = int((x + w/2) * w_img)
                y2 = int((y + h/2) * h_img)
                color = (255, 0, 0) if class_id == 0 else (0, 255, 0)
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, "Car" if class_id == 0 else "Empty", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        plt.imshow(img)
        plt.axis("off")
        plt.show()

# Show some training images with labels
visualize_samples("train")

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>